# Quickstart

This notebook is just to make sure that everything works. Firstly, create a file called `.env` in this folder, and add the following content, obviously with your values. There are two ways to authenticate: using a token or using the Azure CLI (AAD authentication). The Azure CLI is the recommended way.

```
OPENAI_API_KEY=xxxxxx
OPENAI_API_BASE=https://xxxxxxx.openai.azure.com/
```

When authenticating using Azure CLI + Azure AD, open a terminal and run the following command: `az login --use-device-code`. This will open a browser window where you can authenticate. After authentication, use Option 1 code to run and you are good to go.

*(You could as well run the following command to get the token: `az account get-access-token --resource https://management.core.windows.net/ --query accessToken -o tsv`. Copy the token and add it to the `.env` file and then use the token to authenticate.)*

> **🔔 Please note that in all following notebooks you will need to set the authentication method accordingly!**

In [7]:
import os
import tiktoken
import openai
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from openai.embeddings_utils import cosine_similarity
from tenacity import retry, wait_random_exponential, stop_after_attempt

# Load environment variables
load_dotenv()

# Option 1 - Use Azure AD authentication with az cli (use az login in terminal)
#default_credential = DefaultAzureCredential()
#token = default_credential.get_token("https://cognitiveservices.azure.com/.default")
#openai.api_type = "azure_ad"
#openai.api_base = os.environ.get("OPENAI_API_BASE")
#openai.api_key = token.token
#openai.api_version = "2022-12-01"

# Option 2 - Using Access Key
openai.api_type = "azure"
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = "2022-12-01"

# Define embedding model and encoding
EMBEDDING_MODEL = 'text-embedding-ada-002'
COMPLETION_MODEL = 'text-davinci-003'
encoding = tiktoken.get_encoding('cl100k_base')

Test if completitions work:

In [8]:
response = openai.Completion.create(engine="text-davinci-003",
                                    prompt="Knock knock.",
                                    temperature=0)
print(response.choices[0].text)



Who's there?

Boo.

Boo who?



Do it in a streaming fashion:

In [9]:
import sys
for resp in openai.Completion.create(engine='text-davinci-003', prompt='Give me 10 taglines for an ice cream shop', max_tokens=512, stream=True):
    sys.stdout.write(resp.choices[0].text)
    sys.stdout.flush()



1. Deliciously cool treats for every taste! 
2. Stop in for a scoop of happiness! 
3. All the flavor you need for an ice cream treat! 
4. The sweetest way to beat the heat. 
5. Treat yourself to something special today! 
6. Cool down with a scoop of our amazing ice creams! 
7. Every scoop is a happy moment!
8. Let us tantalize your taste buds today!
9. No matter your flavor, we've got you covered! 
10. The tastiest way to satisfy a sweet tooth!

Test if embeddings work:

In [10]:
e = openai.Embedding.create(input="Hello World!", engine=EMBEDDING_MODEL)["data"][0]["embedding"]
print(e)

[0.0023989276960492134, 0.00032934165210463107, -0.0022370079532265663, -0.02578139863908291, -0.011444425210356712, 0.0010194656206294894, -0.01477714255452156, 0.0034081749618053436, 6.180068157846108e-05, -0.02761753648519516, 0.02388237789273262, 0.005002220161259174, -0.02756723202764988, -0.010174219496548176, 0.007784724701195955, 0.011683374643325806, 0.02506454847753048, -0.014148328453302383, 0.007313113659620285, 0.009520253166556358, -0.007099316455423832, 0.00853930227458477, 0.01028111856430769, 0.005747365299612284, -0.006024043541401625, 0.001855003065429628, 0.004973923321813345, -0.01886443793773651, 0.03747734799981117, -0.024108750745654106, 0.016047347337007523, -0.012167561799287796, -0.003197522135451436, -0.024561498314142227, 0.009935270063579082, -0.011884595267474651, 0.002636305056512356, -0.012519698590040207, 0.015355652198195457, -0.018575182184576988, 0.008709082379937172, 0.00969632063060999, 0.004848160315304995, -0.01370815746486187, -0.04089809954166

Test if tokenizer works:

In [11]:
tokens = encoding.encode("Hello world!")
print(tokens)
print(len(tokens))

[9906, 1917, 0]
3


Also let's check if LangChain works:

In [6]:
import os
import openai
from dotenv import load_dotenv
from langchain.llms import AzureOpenAI
from langchain.embeddings import OpenAIEmbeddings

# Load environment variables
load_dotenv()

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_version = "2022-12-01"
openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("OPENAI_API_KEY")

# Create a completion
llm = AzureOpenAI(deployment_name="text-davinci-003")
joke = llm("Tell me a dad joke")
print(joke)

# Create embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)
text = "This is a test document."

# Embed a single document
query_result = embeddings.embed_query(text)
print(query_result)

# Embed multiple documents at once
doc_result = embeddings.embed_documents([text, text])

USING API_BASE: 
https://genaihackathon.openai.azure.com/


Q: Did you hear about the restaurant on the moon?
A: Great food, no atmosphere!
[-0.0031265634017012558, 0.011093720429545843, -0.00404104577150244, -0.011746683703689701, -0.0009627883239309025, 0.010767237861151371, -0.010394116522396335, -0.0052736814265254535, -0.009874410603144264, -0.02617185104404581, 0.02033515580298649, 0.022547236834886564, -0.00750908277492835, 0.01728354896491996, -0.005976616674415002, 0.019175811972550725, 0.02126796101251917, -0.015631151803797528, 0.007655666424102841, -0.018416242069435405, -0.0006825303201955304, -0.006453013767062734, -0.010960462143331514, 0.017963164827629226, -0.02216078827052628, -0.0030899174894076327, 0.014405179295915869, -0.029983027785227667, 0.018482870746881295, -0.007962159551073256, 0.010060971272521781, -0.019415675956413976, -0.0036579292463741037, -0.024266262673454886, -0.005530203045411449, 0.0038411590406728533, -0.005706769692568846, -0.029476647228935757